In [6]:
import cv2
import numpy as np

# 定义相机网格的大小
camera_H = 16  # 高度（相机垂直方向上的数量）
camera_W = 16  # 宽度（相机水平方向上的数量）
camera_Tot = camera_W * camera_H  # 总的相机数量
image_Name = [""] * (camera_Tot)  # 用于存储所有图像文件路径的列表
image = [None] * (camera_Tot)  # 用于存储所有图像的矩阵

def read_Image():
    """
    读取所有相机视角对应的图像文件，并存储在 'image' 列表中。
    使用 OpenCV 读取图片，如果文件不存在或无法读取，则返回 False。
    """
    global image
    for i in range(camera_Tot):  # 遍历所有相机视角
        # 根据文件编号格式生成文件路径名称
        if i < 9:
            image_Name[i] = f"./toyLF/lowtoys00{i+1}.bmp"
        elif i < 99:
            image_Name[i] = f"./toyLF/lowtoys0{i+1}.bmp"
        else:
            image_Name[i] = f"./toyLF/lowtoys{i+1}.bmp"

    # 使用 OpenCV 逐个读取每个图像文件
    for i in range(camera_Tot):
        readimg = cv2.imread(image_Name[i])  # 读取图像
        if readimg is None:  # 如果读取失败，输出错误信息
            print(f"Could not open or find the image: {image_Name[i]}")
            return False
        image[i] = readimg.astype(np.float64) / 255.0  # 将图像像素归一化至 [0, 1] 范围内
    return True

In [7]:
read_Image()

True

In [12]:
camera_Main_Pos = (0.52, 0.52)  # 主相机位置（位于网格中央）
import math

def get_Normal_pdf(x, m, s):
    """
    计算正态分布概率密度函数，用于距离权重的计算。
    x: 距离
    m: 均值
    s: 标准差
    返回值为正态分布的概率密度。
    """
    inv_sqrt_2pi = 0.3989422804014327  # 1 / sqrt(2*pi)
    a = (x - m) / s  # 计算标准化值
    return inv_sqrt_2pi / s * np.exp(-0.5 * a * a)  # 返回正态分布的概率密度值

def advance_Image():
    """
    使用基于高斯加权插值的方式生成最终合成图像。
    """
    global image_Final
    image_Final = np.zeros_like(image[0])  # 初始化输出图像
    height, width = image_Final.shape[:2]  # 获取图像尺寸

    # 计算主相机位置在网格中的浮点坐标
    y = 1 + camera_Main_Pos[1] * (camera_H - 1)
    x = 1 + camera_Main_Pos[0] * (camera_W - 1)
    points = []  # 存储有效的相机视角
    weights = []  # 存储每个相机的权重

    # 计算每个相机视角的权重
    for a in range(1, camera_W + 1):
        for b in range(1, camera_H + 1):
            num = (b - 1) * camera_W + a  # 当前相机的编号
            dis = math.sqrt((x - a) ** 2 + (y - b) ** 2)  # 计算距离
            w = get_Normal_pdf(dis, 0.0, 1.0)  # 计算正态分布的权重
            if w > 0.0001:  # 忽略小权重
                dx = int((a - x) * f * width / z2)  # x方向上的位移
                dy = int(-(b - y) * f * height / z1)  # y方向上的位移
                points.append((num, dx, dy))  # 存储相机索引和位移
                weights.append(w)  # 存储权重

    # 对每个像素进行插值计算
    for i in range(height):
        for j in range(width):
            total_weight = sum(weights)  # 总权重
            if total_weight < 1e-8:  # 如果总权重太小，跳过该像素
                continue

            # 归一化权重
            normalized_weights = [w / total_weight for w in weights]
            for idx, (num, dx, dy) in enumerate(points):
                if 0 <= i + dy < height and 0 <= j + dx < width:  # 确保像素位置有效
                    image_Final[i, j] += normalized_weights[idx] * image[num][i + dy, j + dx]  # 插值计算

[array([[[0.23529412, 0.21568627, 0.16078431],
         [0.23137255, 0.21176471, 0.16078431],
         [0.23137255, 0.21568627, 0.16470588],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],
 
        [[0.22352941, 0.22352941, 0.19215686],
         [0.21568627, 0.21960784, 0.19607843],
         [0.21568627, 0.22745098, 0.20392157],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],
 
        [[0.26666667, 0.26666667, 0.20392157],
         [0.2627451 , 0.25490196, 0.2       ],
         [0.2627451 , 0.25882353, 0.20392157],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],
 
        ...,
 
        [[0.25882353, 0.27843137, 0.36862745],
         [0.27058824, 0.30588235, 0.43529